In [29]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Attention
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [13]:
# in order to make sure that all the input images are of same scale, we are using ImageDataGenerator (for data augmentation)
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   vertical_flip = True,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('output/train/',
                                                 target_size = (32, 32),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 927 images belonging to 2 classes.


In [14]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('output/val',
                                            target_size = (32, 32),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 330 images belonging to 2 classes.


In [30]:
inputs = Input((32, 32, 3))

conv1 = Conv2D(filters = 16, kernel_size = (3, 3), activation = 'relu', padding='same')(inputs)
maxpool1 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv1)

# attn1 = Attention(maxpool1, ))

conv2 = Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool1)
maxpool2 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv2)

conv3 = Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool2)
maxpool3 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv3)

attn = Attention()([maxpool3, maxpool3])

flattened = Flatten()(attn)

fc1 = Dense(units = 128, activation = 'relu')(flattened)
drop1 = Dropout(rate = 0.5)(fc1)

final = Dense(units = 1,activation='sigmoid')(drop1)

model = Model(inputs=inputs, outputs=final)

In [32]:
model.compile(optimizer = 'adadelta',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

In [34]:
model.fit(x = training_set, validation_data = test_set, epochs = 1)

29/29 [==============================] - 186s 6s/step - loss: 0.7048 - accuracy: 0.4768 - val_loss: 0.7015 - val_accuracy: 0.4515
